# Import packages:

In [17]:
from openbiolink.obl2021 import OBL2021Dataset
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data
import torch
import math

# Accessing Data:

In [4]:
obl_dataset = OBL2021Dataset()

Dataset found in D:\programming repo-1\university of glasgow\msc project\MSc.Project\obl2021, omitting download...


In [5]:
# get the training, validation and test sets

#obl_train_dataset = obl_dataset.training
#obl_valid_dataset = obl_dataset.validation
#obl_test_dataset = obl_dataset.testing

In [6]:
#checking out the tensor shape
print('The shape of the training tensor is : ', obl_dataset.training.shape)
print('The shape of the validation tensor is : ', obl_dataset.validation.shape)
print('The shape of the testing tensor is : ', obl_dataset.testing.shape)

#checking out the tensor dtype
print('The data type of the training tensor is : ', obl_dataset.training.dtype)
print('The data type of the validation tensor is : ', obl_dataset.validation.dtype)
print('The data type of the testing tensor is : ', obl_dataset.testing.dtype)

The shape of the training tensor is :  torch.Size([4192002, 3])
The shape of the validation tensor is :  torch.Size([186301, 3])
The shape of the testing tensor is :  torch.Size([180964, 3])
The data type of the training tensor is :  torch.int64
The data type of the validation tensor is :  torch.int64
The data type of the testing tensor is :  torch.int64


# Embedding Models:

## TransE:

In [122]:
class TransE(nn.Module):
    def __init__(self, device, num_entity, num_relation, emb_dim, gamma):
        super(TransE, self).__init__()
        self.device = device
        self.emb_dim = emb_dim
        self.num_entity = num_entity
        self.num_relation = num_relation
        
        #initialize entity embeddings
        self.entity_emb = self.initialize_emb(self.num_entity, self.emb_dim)

        #initialie relation embeddings
        self.relation_emb = self.initialize_emb(self.num_relation, self.emb_dim)
        self.relation_emb.weight.data.div_(self.relation_emb.weight.data.norm(p=2, dim=1, keepdim=True))
        #create the loss function
        self.loss_fn = nn.MarginRankingLoss(margin=gamma)
        

    def initialize_emb(self, num_emb, emb_dim):
        emb_weight_range = 6 / math.sqrt(emb_dim)
        emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=emb_dim, device=self.device)
        emb.weight.data.uniform_( -emb_weight_range, emb_weight_range )
        return emb
    
    def forward(self, pos_triplet, neg_triplet):
        self.entity_emb.weight.data.div_(self.entity_emb.weight.data.norm(p=2, dim=1, keepdim=True))
        pos_distance = self.cal_distance(pos_triplet)
        neg_distance = self.cal_distance(neg_triplet)
        return self.loss_fn(pos_distance, neg_distance, torch.tensor([[-1]]*512, dtype=torch.int64, device=self.device))
    
    def cal_distance(self, triplet):
        head = triplet[:,0]
        relation = triplet[:,1]
        tail = triplet[:,2]
        #print(head.shape)
        return (self.entity_emb(head) + self.relation_emb(relation) - self.entity_emb(tail))
       
        

In [114]:
#test transe_model, remove later
#transe_model = TransE(device = 'cuda', num_entity = 100, num_relation = 10, emb_dim = 100, gamma = 0.01)

### Create Dataset:

In [113]:
'''
class Create_dataset(torch.utils.data.IterableDataset):
    def __init__(self, data):
        self.data = data
        self.size = data.size()
    
    def __iter__(self):
        return iter(self.data)
    
    def size(self):
        return self.size

obl_train_dataset = TensorDataset(obl_dataset.training)
obl_valid_dataset = TensorDataset(obl_dataset.validation)
obl_test_dataset = TensorDataset(obl_dataset.testing)

dataset = Create_dataset(obl_dataset.training)

print(type(obl_dataset.training.shape[0]))
print(obl_dataset.training.size())
'''

'''
count = 0
for i in dataset:
    print(i)
    count +=1
    if count == 10:
        break
'''

'\ncount = 0\nfor i in dataset:\n    print(i)\n    count +=1\n    if count == 10:\n        break\n'

### Create DataLoader:

In [90]:
def create_dataloader(dataset, batch_size, shuffle):
    tensor_dataset = TensorDataset(dataset)
    return DataLoader(tensor_dataset, batch_size=batch_size, shuffle=shuffle) 

### Training Function:

In [115]:
def train_transe(model, data_loader, optimizer, epoch, device):
    for i in range(epoch):
        for index, batch_data in enumerate(data_loader):  
            #print(len(batch_data))
            #print(batch_data[0][0])
            #break
            optimizer.zero_grad()
            sample_data = batch_data[0].to(device)
            corr_sample_data = None
            loss = model(sample_data, sample_data)
            loss.mean().backward()
            optimizer.step()

### Evaluation Function:

In [116]:
def evaluate_model():
    pass

### Set parameters and call train TransE:

In [123]:
transe_model = TransE(device = 'cuda', num_entity = 184732, num_relation = 28, emb_dim = 100, gamma = 0.01)
optimizer = torch.optim.Adam(transe_model.parameters(), lr=0.0001)

data_loader = create_dataloader(obl_dataset.training, batch_size=512, shuffle=True)
train_transe(model=transe_model, data_loader=data_loader, optimizer=optimizer, epoch=20, device='cuda')

RuntimeError: The size of tensor a (512) must match the size of tensor b (258) at non-singleton dimension 0